In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [2]:
conf = SparkConf()
conf.set('spark.driver.bindAddress', '0.0.0.0')
conf.set('spark.driver.host', '0.0.0.0')

In [3]:
sc = SparkContext(master='local', appName='miPrimerContexto', conf=conf)

21/10/14 00:15:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
rdd1 = sc.parallelize([x for x in range(1, 100)])
type(rdd1)

pyspark.rdd.RDD

In [5]:
rdd1.take(5)

[1, 2, 3, 4, 5]

In [6]:
sc

<SparkContext master=local appName=miPrimerContexto>

In [7]:
!ls files/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista.csv	 evento.csv	      paises.csv
deportista2.csv  juegos.csv	      resultados.csv


In [8]:
files_path = 'files/'

In [9]:
equipos_olimpicos_rdd = sc.textFile(f'{files_path}paises.csv') \
    .map(lambda line: line.split(','))

In [10]:
equipos_olimpicos_rdd.take(5)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG']]

In [11]:
equipos_olimpicos_rdd.map(lambda x: (x[2])).distinct().count()

231

In [12]:
equipos_olimpicos_rdd.map(lambda x: (x[2], x[1])).groupByKey().mapValues(len).take(15)

[('sigla', 1),
 ('AUT', 11),
 ('MEX', 9),
 ('ARG', 18),
 ('AFG', 1),
 ('IRL', 7),
 ('SUI', 17),
 ('ALB', 1),
 ('POR', 21),
 ('FRA', 155),
 ('ITA', 36),
 ('ALG', 1),
 ('SWE', 52),
 ('URS', 16),
 ('AUS', 23)]

In [13]:
equipos_olimpicos_rdd.map(lambda x: (x[2], x[1])).groupByKey().mapValues(len).take(5) # I can replace in maxValues the parameter by min, list, tuple, set, etc.

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [14]:
equipos_olimpicos_rdd.map(lambda x: (x[2], x[1])).groupByKey().mapValues(list).take(5)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl']),
 ('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan'])]

In [15]:
equipos_argentinos = equipos_olimpicos_rdd.filter(lambda x: "ARG" in x)

In [16]:
equipos_argentinos.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [17]:
deportistas_olimpicos_rdd = sc.textFile(f'{files_path}deportista.csv')
deportistas_olimpicos_rdd2 = sc.textFile(f'{files_path}deportista2.csv')

In [18]:
deportista_olimpico_rdd = deportistas_olimpicos_rdd.union(deportistas_olimpicos_rdd2)

In [20]:
deportista_olimpico_rdd.count()

135572

In [21]:
equipos_olimpicos_rdd.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [22]:
deportista_olimpico_rdd.top(2)

['deportista_id,nombre,genero,edad,altura,peso,equipo_id',
 '99999,Alexander Grant Alick Rennie,1,32,182,71,967']

In [23]:
deportista_olimpico_rdd.map(lambda x: [x[-1], x[:-1]]) \
    .join(equipos_olimpicos_rdd.map(lambda x: [x[0], x[2]])) \
    .takeSample(False, 6, 25) # repeated=False, number_of_samples=6, seed_number=25

[('2', ('54867,Ral Jimeno Prez,1,25,186,100,98', 'MEX')),
 ('4', ('13586,Tatyana Aleksandrovna Borodulina,2,21,160,52,87', 'ARG')),
 ('7', ('93049,Flavia Pennetta,2,26,172,58,50', 'SUI')),
 ('8', ('19512,Cludia Silvia Chabalgoity,2,21,0,0,13', 'ALB')),
 ('4', ('60478,Clyde Edward Eddie King,1,20,0,0,17', 'ARG')),
 ('5', ('25977,George Franois Frans de Bruijn Kops,1,21,0,0,70', 'AFG'))]

In [24]:
resultados = sc.textFile(f'{files_path}resultados.csv')

In [25]:
resultados.take(5)

['resultado_id,medalla,deportista_id,juego_id,evento_id',
 '1,NA,1,39,1',
 '2,NA,2,49,2',
 '3,NA,3,7,3',
 '4,Gold,4,2,4']